# CryptoCurrency Portfolio Tracker in Python
## Part 3: Creating a Full-Fledged Application

In [16]:
import pandas as pd
import sqlite3
import os

def get_connection():
    conn = sqlite3.connect('../Application/crypto_portfolio.db')
    return conn

In [ ]:
conn = get_connection()
cursor = conn.cursor()
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS prices (
    symbol TEXT,
    price REAL,
    volume_24h REAL,
    volume_change_24h REAL,
    percent_change_1h REAL,
    percent_change_24h REAL,
    percent_change_7d REAL,
    percent_change_30d REAL,
    percent_change_60d REAL,
    percent_change_90d REAL,
    market_cap REAL,
    market_cap_dominance REAL,
    fully_diluted_market_cap REAL,
    tvl REAL,
    last_updated TEXT
)
''')
conn.commit()
conn.close()

In [5]:
conn = get_connection()
prices = pd.read_sql("SELECT * FROM prices;",con=conn)
display(prices)
conn.close()

,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated


### Get Crypto Prices from API

In [6]:
import pandas as pd
import sqlite3
import os
api_key = os.environ.get('CMC_API_KEY')


def get_response_multiple(symbols):
    from requests import Session
    import json
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    session = Session()
    session.headers.update(headers)
    parameters = {
        'symbol': ','.join(symbols)
    }
    response = session.get(url, params=parameters)
    return json.loads(response.text)



def clean_response_multiple(symbols):
    data = get_response_multiple(symbols)
    df = pd.DataFrame(
        [{'symbol': symbol, **data['data'][symbol][0]['quote']['USD']} for symbol in symbols]
    )
    return df

In [7]:
df = clean_response_multiple(['BTC','ETH'])

In [8]:
df

,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated
0,BTC,67715.421223,2.226929e+10,-17.3671,0.139538,-1.384677,-2.013076,15.496249,3.757425,8.789458,1.334425e+12,52.7119,1.422024e+12,None,2024-06-01T12:57:00.000Z
1,ETH,3795.867127,1.289180e+10,-10.1754,0.240584,-0.758731,1.573154,27.273663,15.042442,10.864338,4.560434e+11,18.0144,4.560434e+11,None,2024-06-01T12:58:00.000Z


In [9]:
for i in df.columns:
    print(type(df[i].iloc[0]))

<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'NoneType'>
<class 'str'>


In [10]:
conn = get_connection()
cursor = conn.cursor()
for _, row in df.iterrows():
        cursor.execute(f'''
        insert into prices (symbol, price, volume_24h, volume_change_24h, percent_change_1h, percent_change_24h, 
                                percent_change_7d, percent_change_30d, percent_change_60d, percent_change_90d, 
                                market_cap, market_cap_dominance, fully_diluted_market_cap, tvl, last_updated)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['symbol'], row['price'], row['volume_24h'], row['volume_change_24h'], row['percent_change_1h'],
            row['percent_change_24h'], row['percent_change_7d'], row['percent_change_30d'],
            row['percent_change_60d'], row['percent_change_90d'], row['market_cap'],
            row['market_cap_dominance'], row['fully_diluted_market_cap'], row['tvl'], row['last_updated']))
conn.commit()  
conn.close()

In [11]:
conn = get_connection()
cursor = conn.cursor()
prices = pd.read_sql("select * from prices", con=conn)   
conn.close()
display(prices)

,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated
0,BTC,67715.421223,2.226929e+10,-17.3671,0.139538,-1.384677,-2.013076,15.496249,3.757425,8.789458,1.334425e+12,52.7119,1.422024e+12,None,2024-06-01T12:57:00.000Z
1,ETH,3795.867127,1.289180e+10,-10.1754,0.240584,-0.758731,1.573154,27.273663,15.042442,10.864338,4.560434e+11,18.0144,4.560434e+11,None,2024-06-01T12:58:00.000Z


In [3]:
def exec_sql(query):
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute(query)
    conn.commit()
    conn.close()

In [4]:
# exec_sql('delete from prices')

In [26]:
# conn = get_connection()
# df.to_sql(prices, con=conn, if_exists='replace', index=False)
# conn.close()

In [12]:
def update_prices():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('delete from prices')
    results = cursor.execute("select distinct symbol from portfolio").fetchall()
    coins = [row[0] for row in results]
    df = clean_response_multiple(coins)
    for _, row in df.iterrows():
        cursor.execute(f'''
        insert into prices (symbol, price, volume_24h, volume_change_24h, percent_change_1h, percent_change_24h, 
                                percent_change_7d, percent_change_30d, percent_change_60d, percent_change_90d, 
                                market_cap, market_cap_dominance, fully_diluted_market_cap, tvl, last_updated)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['symbol'], row['price'], row['volume_24h'], row['volume_change_24h'], row['percent_change_1h'],
            row['percent_change_24h'], row['percent_change_7d'], row['percent_change_30d'],
            row['percent_change_60d'], row['percent_change_90d'], row['market_cap'],
            row['market_cap_dominance'], row['fully_diluted_market_cap'], row['tvl'], row['last_updated']))
    conn.commit()
    conn.close()
update_prices()

In [13]:
conn = get_connection()
dfjoin = pd.read_sql('select portfolio.amount, portfolio.cost, prices.* from portfolio left join prices on portfolio.symbol = prices.symbol', conn)
conn.close()
dfjoin

,amount,cost,symbol,price,volume_24h,volume_change_24h,percent_change_1h,percent_change_24h,percent_change_7d,percent_change_30d,percent_change_60d,percent_change_90d,market_cap,market_cap_dominance,fully_diluted_market_cap,tvl,last_updated
0,1.0,45000.0,BTC,67683.368855,2.224130e+10,-17.4554,0.105889,-1.343480,-2.073856,15.606000,3.598486,8.801129,1.333794e+12,52.6896,1.421351e+12,None,2024-06-01T13:00:00.000Z
1,5.0,10000.0,ETH,3794.443815,1.287375e+10,-10.3927,0.219079,-0.731952,1.462515,27.257003,15.133826,10.830890,4.558724e+11,18.0077,4.558724e+11,None,2024-06-01T12:59:00.000Z


### PostProcessing the data

In [15]:
dfjoin.columns

Index(['amount', 'cost', 'symbol', 'price', 'volume_24h', 'volume_change_24h',
       'percent_change_1h', 'percent_change_24h', 'percent_change_7d',
       'percent_change_30d', 'percent_change_60d', 'percent_change_90d',
       'market_cap', 'market_cap_dominance', 'fully_diluted_market_cap', 'tvl',
       'last_updated'],
      dtype='object')

### All Together

In [ ]:
import pandas as pd
import sqlite3
import os
api_key = os.environ.get('CMC_API_KEY')

def get_connection():
    conn = sqlite3.connect('crypto_portfolio.db')
    return conn

def get_response_multiple(symbols):
    from requests import Session
    import json
    url = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest'
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': api_key,
    }
    session = Session()
    session.headers.update(headers)
    parameters = {
        'symbol': ','.join(symbols)
    }
    response = session.get(url, params=parameters)
    return json.loads(response.text)

def clean_response_multiple(symbols):
    data = get_response_multiple(symbols)
    df = pd.DataFrame(
        [{'symbol': symbol, **data['data'][symbol][0]['quote']['USD']} for symbol in symbols]
    )
    return df

def update_prices():
    conn = get_connection()
    cursor = conn.cursor()
    cursor.execute('delete from prices')
    results = cursor.execute("select distinct symbol from portfolio").fetchall()
    coins = [row[0] for row in results]
    df = clean_response_multiple(coins)
    for _, row in df.iterrows():
        cursor.execute(f'''
        insert into prices (symbol, price, volume_24h, volume_change_24h, percent_change_1h, percent_change_24h, 
                                percent_change_7d, percent_change_30d, percent_change_60d, percent_change_90d, 
                                market_cap, market_cap_dominance, fully_diluted_market_cap, tvl, last_updated)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (row['symbol'], row['price'], row['volume_24h'], row['volume_change_24h'], row['percent_change_1h'],
            row['percent_change_24h'], row['percent_change_7d'], row['percent_change_30d'],
            row['percent_change_60d'], row['percent_change_90d'], row['market_cap'],
            row['market_cap_dominance'], row['fully_diluted_market_cap'], row['tvl'], row['last_updated']))
    conn.commit()
    conn.close()
update_prices()

### Data Visualization

In [27]:
conn = get_connection()
raw_df = pd.read_sql('select portfolio.amount, portfolio.cost, prices.* from portfolio left join prices on portfolio.symbol = prices.symbol', conn)

In [36]:
from datetime import datetime
last_update = datetime.strptime(min(raw_df['last_updated']), '%Y-%m-%dT%H:%M:%S.%fZ').strftime('%Y-%m-%d %I:%M %p')

# datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')


# formatted_datetime = last_update.strftime('%Y-%m-%d %I:%M %p')

print(last_update)

2024-06-02 02:49 PM


In [41]:
coins = raw_df[['symbol','amount','cost','price','percent_change_24h']].copy()
coins['avg_price'] = coins['cost'] / coins['amount']
coins['value'] = coins['amount']*coins['price']
coins['net'] = coins['value'] - coins['cost']
coins['%'] = (coins['net'] / coins['cost'])*100
coins['X'] = coins['value'] / coins['cost']
coins['24h_value'] = coins['value']/((coins['percent_change_24h']/100)+1)
coins['24h_net'] = coins['value'] - coins['24h_value']

In [30]:
coins.columns

Index(['amount', 'cost', 'symbol', 'price', 'volume_24h', 'volume_change_24h',
       'percent_change_1h', 'percent_change_24h', 'percent_change_7d',
       'percent_change_30d', 'percent_change_60d', 'percent_change_90d',
       'market_cap', 'market_cap_dominance', 'fully_diluted_market_cap', 'tvl',
       'last_updated', 'avg_price', 'value', 'net', '%', 'X'],
      dtype='object')